# Loan pre-payment analysis

Create Date: 08/June/2020
Last updated: 

Home Loan repayment analysis before even starting the EMI


### re-payment schedule

1. private firm till 60 age loan re-payment.
2. Gov job till age 70 repayment possible.


### Loan eligibility check on 08/jun/2020

SBI is ready to give upto 80%.


https://cleartax.in/s/home-loan-tax-benefit

In [1]:
# Basic constants involved.
sqft = 1600
price_sqft = 5100
car_charge = 350000

# Total price of an asset without any tax or other charges.
total_asset_value = sqft * price_sqft + car_charge

# 1% cess extra
gst_f = lambda x: (x / 100) * (7.5 + 1)

land_value_f = lambda total_v: total_v * (1/3)
const_cost_f = lambda total_v: total_v * (2/3)
reg_f = lambda land_v: (land_v / 100 ) * 7.0

gst = gst_f(const_cost_f(total_asset_value))
reg_charge = reg_f(land_value_f(total_asset_value))

In [22]:
land_value_f(total_asset_value)

2836666.6666666665

In [143]:
const_cost_f(total_asset_value)

5673333.333333333

In [158]:
print(f"Gst for construction: {gst}")

Gst for construction: 482233.3333333333


In [34]:
grant_total = total_asset_value + gst + reg_charge + 200000
grant_total

9390800.0

In [20]:
loan_amount = total_asset_value + gst

In [25]:
# loan_amount * (8/10)
loan_amount

8992233.333333334

In [4]:
co_pay = loan_amount / 4
exp_loan_amount = loan_amount * ( 3 /4)

In [19]:
exp_loan_amount

6744175.0

In [66]:
# percentage allocation quarterly for under-construction items.
allocation = [36, 6, 6, 6, 6, 6, 6, 6, 6, 6, 3, 3, 2, 1, 1]

allocation = [16, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]

In [5]:
# Initial allocation of 36%
# 
alloc1 = loan_amount * (36 / 100)
copay1 = alloc1 / 4
loan1 = alloc1 * (3/4)

In [231]:
copay1, loan1, exp_loan_amount

(809301.0, 2427903.0, 6744175.0)

## Co-pay option for on-going projects

In [56]:
def pre_emi_calculation(exp_loan_amount,
          allocation,
          block_cont_ratio=0.25,
          interest=8.0, init_full=False, special=False):
    """Quarter wise interest calculation with co-payment option.
    
    Only interest will be calculated till the building is done.
    
    """
    
    # Special schema based on custom division.
    if special:
        init_block = exp_loan_amount * (16 / 100)
        init_self = 1000000
        init_loan = init_block - init_self
        base_int = (init_loan * (interest / 100)) / 12
    else:
        init_alp = allocation[0]
        init_block = exp_loan_amount * (init_alp / 100)
        init_loan = init_block * (1 - block_cont_ratio)
        init_self = init_block * (block_cont_ratio)
        base_int = (init_loan * (interest / 100)) / 12
    
    if init_full:
        total = 0
        curr_loan = 0
        curr_self = init_block
    else:
        total = base_int
        curr_loan = init_loan
        curr_self = init_self
        
    print(f"Initial Loan borrow Interest: {base_int}")
    print(f"Initial Loan: {curr_loan}, Initial Invest: {curr_self} ")
    
    block_interest = []
    allocs = allocation[1:]
    for alloc in allocs:
        next_blok = exp_loan_amount * (alloc / 100)
        # 75% vs 25%
        next_loan_part = next_blok * (1 - block_cont_ratio)
        next_self_part = next_blok * block_cont_ratio
        curr_self += next_self_part
        curr_loan += next_loan_part
        base_int = (curr_loan * (interest / 100)) / 12
        
        # Monthly payments
        # 1. interest for already dispersed 
        # 2. Accumulate for next installment.
        print(f"Curr Loan: {curr_loan: .2f}, Curr Self: {curr_self : .2f} QLBlock: {next_loan_part :.2f}, Qself({block_cont_ratio :.2f}): {next_self_part : .2f}, QInt: {base_int * 3 : .2f}, MPay: {base_int : .2f}")
        total += base_int * 3
        block_interest.append(base_int * 3)
        
    print(f"\nTotal pre-emi interest: {total}, principle: {curr_loan}")
    return block_interest
    

In [62]:
# 1000000 - (loan_amount * (16 / 100))
loan_amount

8992233.333333334

In [71]:
int_blocks = pre_emi_calculation(
    exp_loan_amount=loan_amount,
    allocation=allocation,
    interest=7.65,
    block_cont_ratio=0.4, special=True)

Initial Loan borrow Interest: 2797.078000000001
Initial Loan: 438757.3333333335, Initial Invest: 1000000 
Curr Loan:  762477.73, Curr Self:  1215813.60 QLBlock: 323720.40, Qself(0.40):  215813.60, QInt:  14582.39, MPay:  4860.80
Curr Loan:  1086198.13, Curr Self:  1431627.20 QLBlock: 323720.40, Qself(0.40):  215813.60, QInt:  20773.54, MPay:  6924.51
Curr Loan:  1409918.53, Curr Self:  1647440.80 QLBlock: 323720.40, Qself(0.40):  215813.60, QInt:  26964.69, MPay:  8988.23
Curr Loan:  1733638.93, Curr Self:  1863254.40 QLBlock: 323720.40, Qself(0.40):  215813.60, QInt:  33155.84, MPay:  11051.95
Curr Loan:  2057359.33, Curr Self:  2079068.00 QLBlock: 323720.40, Qself(0.40):  215813.60, QInt:  39347.00, MPay:  13115.67
Curr Loan:  2381079.73, Curr Self:  2294881.60 QLBlock: 323720.40, Qself(0.40):  215813.60, QInt:  45538.15, MPay:  15179.38
Curr Loan:  2704800.13, Curr Self:  2510695.20 QLBlock: 323720.40, Qself(0.40):  215813.60, QInt:  51729.30, MPay:  17243.10
Curr Loan:  3028520.53,

In [12]:
int_blocks = pre_emi_calculation(
    exp_loan_amount=5450 * 1600,
    allocation=allocation,
    interest=7.65,
    block_cont_ratio=0.250,
    init_full=True
)

Initial Loan borrow Interest: 15009.300000000001
Initial Loan: 0, Initial Invest: 3139200.0 
Curr Loan: 392400.0, Curr Self: 3270000.0 QLBlock: 392400.0, Qself(0.25): 130800.0, QInt: 7504.65, MPay: 2501.5499999999997
Curr Loan: 784800.0, Curr Self: 3400800.0 QLBlock: 392400.0, Qself(0.25): 130800.0, QInt: 15009.3, MPay: 5003.099999999999
Curr Loan: 1177200.0, Curr Self: 3531600.0 QLBlock: 392400.0, Qself(0.25): 130800.0, QInt: 22513.95, MPay: 7504.650000000001
Curr Loan: 1569600.0, Curr Self: 3662400.0 QLBlock: 392400.0, Qself(0.25): 130800.0, QInt: 30018.6, MPay: 10006.199999999999
Curr Loan: 1962000.0, Curr Self: 3793200.0 QLBlock: 392400.0, Qself(0.25): 130800.0, QInt: 37523.25, MPay: 12507.75
Curr Loan: 2354400.0, Curr Self: 3924000.0 QLBlock: 392400.0, Qself(0.25): 130800.0, QInt: 45027.9, MPay: 15009.300000000001
Curr Loan: 2746800.0, Curr Self: 4054800.0 QLBlock: 392400.0, Qself(0.25): 130800.0, QInt: 52532.549999999996, MPay: 17510.85
Curr Loan: 3139200.0, Curr Self: 4185600.0 

In [15]:
# total_asset_value * (36 / 100)
# 5232000.0 * (7.5 / 100) / 12

In [14]:
# 37990.7675 + 15000
# 2200000 / 8

1. If project extends by an month will cause per month interest on the release amount accumulates.
2. If loan plans aren't great, we may stop as going with enough liquid cash helps with the interest handling. May be close to 40% of the total value.
3. Backout if the loan plans aren't that great.
4. 